<a href="https://colab.research.google.com/github/aleks-haksly/leetcode/blob/main/pandas_postgres/pandas_postgres_medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

176. Second Highest Salary

https://leetcode.com/problems/second-highest-salary/description/

```sql
SELECT max(salary) as SecondHighestSalary
FROM (
SELECT
    id, salary,
    dense_rank() OVER (ORDER BY salary DESC) AS rn
FROM Employee e)
WHERE rn = 2
```

In [ ]:
data = [[1, 100], [2, 200], [3, 300]]
employee = pd.DataFrame(data, columns=['id', 'salary']).astype({'id':'int64', 'salary':'int64'})

In [ ]:
def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    result = employee.assign(rn = employee.salary.rank(method='dense', ascending=False)).query("rn == 2")[['salary']].rename({"salary":"SecondHighestSalary"}, axis=1)
    return result.head(1) if result.shape[0] else pd.DataFrame([{"SecondHighestSalary": None}])

177. Nth Highest Salary

https://leetcode.com/problems/nth-highest-salary/description/

```sql
CREATE OR REPLACE FUNCTION NthHighestSalary(N INT) RETURNS TABLE (Salary INT) AS $$
BEGIN
  RETURN QUERY (
    -- Write your PostgreSQL query statement below.
    SELECT max(sub.salary) FROM (
    SELECT
        e.salary,
        dense_rank() OVER (ORDER BY e.salary DESC) as rn
    FROM Employee e) sub
    WHERE sub.rn = N
      
  );
END;
$$ LANGUAGE plpgsql;
```

In [ ]:
data = [[1, 100], [2, 200], [3, 300]]
employee = pd.DataFrame(data, columns=['Id', 'Salary']).astype({'Id':'Int64', 'Salary':'Int64'})

In [ ]:
def nth_highest_salary(employee: pd.DataFrame, N: int) -> pd.DataFrame:
    result = employee.assign(rn = employee.salary.rank(method='dense', ascending=False)).query("rn == @N")[['salary']].rename({"salary":f"getNthHighestSalary({N})"}, axis=1)
    return result.head(1) if result.shape[0] else pd.DataFrame([{f"getNthHighestSalary({N})": None}])

178. Rank Scores

https://leetcode.com/problems/rank-scores/description/

```sql
SELECT
    score,
    dense_rank() OVER (ORDER BY score DESC) as rank
FROM Scores
ORDER BY score DESC
```

In [ ]:
data = [[1, 3.5], [2, 3.65], [3, 4.0], [4, 3.85], [5, 4.0], [6, 3.65]]
scores = pd.DataFrame(data, columns=['id', 'score']).astype({'id':'Int64', 'score':'Float64'})

In [ ]:
def order_scores(scores: pd.DataFrame) -> pd.DataFrame:
    return scores.assign(rank = scores.score.rank(method='dense', ascending=False))[['score','rank']].sort_values(by='score', ascending=False)

180. Consecutive Numbers

https://leetcode.com/problems/consecutive-numbers/

```sql
-- Write your PostgreSQL query statement below
SELECT
    DISTINCT consecutivenums FROM (
SELECT
    num as ConsecutiveNums,
    lag(num) OVER (ORDER BY id) as b,
    lead(num) OVER (ORDER BY id) as a
 FROM Logs) sub
 WHERE ConsecutiveNums = a AND ConsecutiveNums = b
 ```


In [ ]:
data = [[1, 1], [2, 1], [3, 1], [4, 2], [5, 1], [6, 2], [7, 2]]
logs = pd.DataFrame(data, columns=['id', 'num']).astype({'id':'Int64', 'num':'Int64'})

In [ ]:
def consecutive_numbers(logs: pd.DataFrame) -> pd.DataFrame:
    condition = ((logs.num.shift(1) == logs.num) & ((logs.num.shift(-1) == logs.num)))
    return logs[condition]['num'].drop_duplicates().rename('ConsecutiveNums').to_frame()

184. Department Highest Salary

https://leetcode.com/problems/department-highest-salary/description/

```sql
SELECT
    d.name AS Department,
    sub.Employee,
    sub.Salary
FROM (
SELECT
    e.departmentId,
    e.name AS Employee ,
    salary AS Salary,
    RANK() OVER (PARTITION BY departmentId ORDER BY Salary DESC) as rnk
FROM Employee e) sub
LEFT JOIN Department d ON sub.departmentId = d.id
WHERE sub.rnk = 1
```

In [ ]:
data = [[1, 'Joe', 70000, 1], [2, 'Jim', 90000, 1], [3, 'Henry', 80000, 2], [4, 'Sam', 60000, 2], [5, 'Max', 90000, 1]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'departmentId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'departmentId':'Int64'})
data = [[1, 'IT'], [2, 'Sales']]
department = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})

In [ ]:
def department_highest_salary(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    return employee[employee.groupby('departmentId')['salary']\
                    .rank(method='min', ascending=False) == 1].merge(department, how='left', left_on='departmentId', right_on='id')\
                    .rename({"name_y": "Department", "name_x": "Employee", "salary": "Salary"}, axis=1)[["Department", "Employee", "Salary"]]

550. Game Play Analysis IV

https://leetcode.com/problems/game-play-analysis-iv/

```sql
-- Write your PostgreSQL query statement below
WITH t AS (
SELECT
    a.player_id,
    min(a.event_date) + INTERVAL '1 day ' as next_day
FROM Activity a
GROUP BY a.player_id )

SELECT
ROUND(COUNT(a.player_id)*1.0 / (SELECT COUNT(DISTINCT player_id) FROM Activity), 2) AS fraction  
FROM Activity a
JOIN t ON a.player_id = t.player_id AND t.next_day = a.event_date

```

In [ ]:
data = [[1, 2, '2016-03-01', 5], [1, 2, '2016-03-02', 6], [2, 3, '2017-06-25', 1], [3, 1, '2016-03-02', 0], [3, 4, '2018-07-03', 5]]
activity = pd.DataFrame(data, columns=['player_id', 'device_id', 'event_date', 'games_played']).astype({'player_id':'Int64', 'device_id':'Int64', 'event_date':'datetime64[ns]', 'games_played':'Int64'})

In [ ]:
def gameplay_analysis(activity: pd.DataFrame) -> pd.DataFrame:
    next_day = activity.groupby("player_id").event_date.apply(lambda x: x.min() + pd.DateOffset(days=1))
    fraction = round(activity.assign(next_day=activity.player_id.map(next_day)).query("event_date == next_day").player_id.nunique() / activity.player_id.nunique(), 2)
    return pd.DataFrame([fraction], columns=['fraction'], index=[0])

570. Managers with at Least 5 Direct Reports

https://leetcode.com/problems/managers-with-at-least-5-direct-reports/description/

```sql
SELECT
    e1.name
FROM Employee e1
JOIN Employee e2 ON e1.id = e2.managerId
GROUP BY e1.id, e1.name
HAVING count(e2.managerId) >= 5
```

In [ ]:
data = [[101, 'John', 'A', None], [102, 'Dan', 'A', 101], [103, 'James', 'A', 101], [104, 'Amy', 'A', 101], [105, 'Anne', 'A', 101], [106, 'Ron', 'B', 101]]
employee = pd.DataFrame(data, columns=['id', 'name', 'department', 'managerId']).astype({'id':'Int64', 'name':'object', 'department':'object', 'managerId':'Int64'})

In [ ]:
def find_managers(employee: pd.DataFrame) -> pd.DataFrame:
    filtered = employee.groupby('managerId', ).agg(reporting = ('id', 'count')).query('reporting >=5', engine='python')
    return employee[employee.id.isin(filtered.index)][['name']]

585. Investments in 2016

https://leetcode.com/problems/investments-in-2016/description/

```sql
WITH uniqu_ltt AS(
SELECT
    lat::text||' '||lon::text as ltconc
FROM Insurance i
GROUP BY lat::text||' '||lon::text
HAVING count(*) = 1),

repeted_tiv AS (
SELECT
tiv_2015
FROM Insurance i
GROUP BY tiv_2015
HAVING COUNT(tiv_2015) > 1)

SELECT
ROUND(sum(tiv_2016)::NUMERIC, 2) as tiv_2016
FROM Insurance
WHERE tiv_2015 in (SELECT tiv_2015 FROM repeted_tiv) AND lat::text||' '||lon::text IN (SELECT ltconc FROM uniqu_ltt)
```

In [ ]:
data = [[1, 10, 5, 10, 10], [2, 20, 20, 20, 20], [3, 10, 30, 20, 20], [4, 10, 40, 40, 40]]
insurance = pd.DataFrame(data, columns=['pid', 'tiv_2015', 'tiv_2016', 'lat', 'lon']).astype({'pid':'Int64', 'tiv_2015':'Float64', 'tiv_2016':'Float64', 'lat':'Float64', 'lon':'Float64'})

In [ ]:
def find_investments(insurance: pd.DataFrame) -> pd.DataFrame:
    uniqu_ltt  = insurance.assign(ltconc = insurance.lat.astype(str).str.cat(insurance.lon.astype(str), sep=' ')).groupby('ltconc').agg(ltconc=('pid', 'count')).query("ltconc == 1", engine='python').index
    repeted_tiv  = insurance.groupby('tiv_2015').agg(tiv_2015_cnt = ('tiv_2015', 'count')).query("tiv_2015_cnt > 1", engine='python').index
    tiv_2016 = insurance[insurance.tiv_2015.isin(repeted_tiv) & insurance.lat.astype(str).str.cat(insurance.lon.astype(str), sep=' ').isin(uniqu_ltt)].tiv_2016.sum().round(2)
    return pd.DataFrame([tiv_2016], columns=['tiv_2016'], index=[0])

602. Friend Requests II: Who Has the Most Friends

https://leetcode.com/problems/friend-requests-ii-who-has-the-most-friends/description/

```sql
WITH temp as
(SELECT requester_id as id FROM RequestAccepted
UNION ALL
SELECT accepter_id as id FROM RequestAccepted)

SELECT id, COUNT(id) as num FROM temp GROUP BY id
ORDER BY num DESC LIMIT 1
```

In [ ]:
data = [[1, 2, '2016/06/03'], [1, 3, '2016/06/08'], [2, 3, '2016/06/08'], [3, 4, '2016/06/09']]
request_accepted = pd.DataFrame(data, columns=['requester_id', 'accepter_id', 'accept_date']).astype({'requester_id':'Int64', 'accepter_id':'Int64', 'accept_date':'datetime64[ns]'})

In [ ]:
def most_friends(request_accepted: pd.DataFrame) -> pd.DataFrame:
    result = request_accepted.requester_id.to_list() + request_accepted.accepter_id.to_list()
    return pd.DataFrame(result, columns=['id'], index=range(len(result))).value_counts().sort_values(ascending=False).head(1).reset_index(name='num')

In [ ]:
import pandas as pd
from collections import Counter

def most_friends(request_accepted: pd.DataFrame) -> pd.DataFrame:
    c = Counter(request_accepted.requester_id)
    c.update(request_accepted.accepter_id)
    return pd.DataFrame(c.most_common()[:1], columns=['id', 'num'], index=[0])

608. Tree Node

https://leetcode.com/problems/tree-node/description/

```sql
SELECT
    id,
    CASE WHEN p_id IS NULL THEN 'Root'
         WHEN id in (SELECT p_id FROM Tree) THEN 'Inner'
         ELSE 'Leaf' END as "type"
FROM Tree
```

In [ ]:
data = [[1, None], [2, 1], [3, 1], [4, 2], [5, 2]]
tree = pd.DataFrame(data, columns=['id', 'p_id']).astype({'id':'Int64', 'p_id':'Int64'})

In [ ]:
import numpy as np

def tree_node(tree: pd.DataFrame) -> pd.DataFrame:
    tree["type"] = np.where(tree.p_id.isna(), 'Root',
                        np.where(~tree.id.isin(tree.p_id), 'Leaf', 'Inner'))
    return tree[['id', 'type']]

626. Exchange Seats

https://leetcode.com/problems/exchange-seats/description/

```sql
SELECT
id,
CASE WHEN id % 2 = 1 THEN LEAD(student, 1, student) OVER (ORDER BY id)
    ELSE LAG(student, 1, student) OVER (ORDER BY id) END AS student
FROM Seat
```

In [ ]:
data = [[1, 'Abbot'], [2, 'Doris'], [3, 'Emerson'], [4, 'Green'], [5, 'Jeames']]
seat = pd.DataFrame(data, columns=['id', 'student']).astype({'id':'Int64', 'student':'object'})

In [ ]:
def exchange_seats(seat: pd.DataFrame) -> pd.DataFrame:
    seat['student'] = np.where((seat.id %2 == 1) & (seat.id != seat.shape[0]), seat.student.shift(-1),
                               np.where(seat.id %2 == 0, seat.student.shift(1), seat.student))
    return seat

1045. Customers Who Bought All Products

https://leetcode.com/problems/customers-who-bought-all-products/description/

```sql
SELECT
    customer_id
FROM Customer
GROUP BY customer_id
HAVING ARRAY_LENGTH(ARRAY_AGG(DISTINCT product_key), 1) = (SELECT COUNT(*) FROM Product);

```

In [ ]:
data = [[1, 5], [2, 6], [3, 5], [3, 6], [1, 6]]
customer = pd.DataFrame(data, columns=['customer_id', 'product_key']).astype({'customer_id':'Int64', 'product_key':'Int64'})
data = [[5], [6]]
product = pd.DataFrame(data, columns=['product_key']).astype({'product_key':'Int64'})

In [ ]:
def find_customers(customer: pd.DataFrame, product: pd.DataFrame) -> pd.DataFrame:
    all_product = set(product.product_key.values)
    return customer.groupby('customer_id')\
                   .product_key\
                   .apply(lambda x: set(x))\
                   .reset_index(name='product_keys')\
                   .query('product_keys == @all_product')\
                   [['customer_id']]

1070. Product Sales Analysis III

https://leetcode.com/problems/product-sales-analysis-iii/description/

```sql
-- Write your PostgreSQL query statement below
WITH t as (
SELECT
    product_id,
    min(year) as min_year
FROM Sales
GROUP BY product_id)

SELECT
    s.product_id,
    s.year as first_year ,
    s.quantity,
    s.price
FROM Sales s
JOIN t ON t.product_id = s.product_id AND t.min_year = s.year
```

In [ ]:
data = [[1, 100, 2008, 10, 5000], [2, 100, 2009, 12, 5000], [7, 200, 2011, 15, 9000]]
sales = pd.DataFrame(data, columns=['sale_id', 'product_id', 'year', 'quantity', 'price']).astype({'sale_id':'Int64', 'product_id':'Int64', 'year':'Int64', 'quantity':'Int64', 'price':'Int64'})
data = [[100, 'Nokia'], [200, 'Apple'], [300, 'Samsung']]
product = pd.DataFrame(data, columns=['product_id', 'product_name']).astype({'product_id':'Int64', 'product_name':'object'})

In [ ]:
def sales_analysis(sales: pd.DataFrame, product: pd.DataFrame) -> pd.DataFrame:
    return sales.assign(min_year = sales.groupby('product_id').year.rank(method='min'))\
                .query('min_year ==  1', engine='python')\
                .iloc[:,1:-1]\
                .rename({'year': 'first_year'}, axis=1)

1158. Market Analysis I

https://leetcode.com/problems/market-analysis-i/description/

```sql
SELECT
    u.user_id as buyer_id,
    u.join_date,
    count(o.order_id) as orders_in_2019
FROM Users u
LEFT JOIN Orders o ON u.user_id = o.buyer_id AND date_part('year', o.order_date) = 2019
GROUP BY u.user_id, u.join_date
```

In [ ]:
data = [[1, '2018-01-01', 'Lenovo'], [2, '2018-02-09', 'Samsung'], [3, '2018-01-19', 'LG'], [4, '2018-05-21', 'HP']]
users = pd.DataFrame(data, columns=['user_id', 'join_date', 'favorite_brand']).astype({'user_id':'Int64', 'join_date':'datetime64[ns]', 'favorite_brand':'object'})
data = [[1, '2019-08-01', 4, 1, 2], [2, '2018-08-02', 2, 1, 3], [3, '2019-08-03', 3, 2, 3], [4, '2018-08-04', 1, 4, 2], [5, '2018-08-04', 1, 3, 4], [6, '2019-08-05', 2, 2, 4]]
orders = pd.DataFrame(data, columns=['order_id', 'order_date', 'item_id', 'buyer_id', 'seller_id']).astype({'order_id':'Int64', 'order_date':'datetime64[ns]', 'item_id':'Int64', 'buyer_id':'Int64', 'seller_id':'Int64'})
data = [[1, 'Samsung'], [2, 'Lenovo'], [3, 'LG'], [4, 'HP']]
items = pd.DataFrame(data, columns=['item_id', 'item_brand']).astype({'item_id':'Int64', 'item_brand':'object'})

In [ ]:
def market_analysis(users: pd.DataFrame, orders: pd.DataFrame, items: pd.DataFrame) -> pd.DataFrame:
    buyers = orders[orders.order_date.dt.year == 2019]\
                  .groupby('buyer_id')\
                  .order_id.count()\
                  .reset_index(name='orders_in_2019')
    return users.merge(buyers, how='left', left_on='user_id', right_on='buyer_id')[['user_id', 'join_date', 'orders_in_2019']].fillna({'orders_in_2019': 0})\
                            .rename({'user_id': 'buyer_id'}, axis=1)

1164. Product Price at a Given Date

https://leetcode.com/problems/product-price-at-a-given-date/description/

```sql
WITH t as (
SELECT
    s1.product_id,
    s2.new_price,
    row_number() OVER (PARTITION BY s1.product_id ORDER BY s2.change_date DESC NULLS LAST) as rn
FROM Products s1
LEFT JOIN  Products s2 ON s1.product_id = s2.product_id AND s1.change_date = s2.change_date  AND s2.change_date <= '2019-08-16')

SELECT
    product_id,
    COALESCE(new_price, 10) AS  price
FROM t
WHERE rn = 1
```

In [ ]:
data = [[1, 20, '2019-08-14'], [2, 50, '2019-08-14'], [1, 30, '2019-08-15'], [1, 35, '2019-08-16'], [2, 65, '2019-08-17'], [3, 20, '2019-08-18']]
products = pd.DataFrame(data, columns=['product_id', 'new_price', 'change_date']).astype({'product_id':'Int64', 'new_price':'Int64', 'change_date':'datetime64[ns]'})

In [ ]:
def price_at_given_date(products: pd.DataFrame) -> pd.DataFrame:
    temp = products[products.change_date.le('2019-08-16')]\
      .assign(max_date = products[products.change_date.le('2019-08-16')]\
              .groupby('product_id')\
              .change_date.rank(method='min', ascending=False))\
              .query("max_date == 1")\
              .set_index('product_id')\
              .new_price
    products['price'] = products.product_id\
                                    .map(temp)\
                                    .fillna(10)
    return products[['product_id','price']].drop_duplicates()

1174. Immediate Food Delivery II

https://leetcode.com/problems/immediate-food-delivery-ii/description/

```sql

WITH t as (
SELECT
    *,
    row_number() OVER (PARTITION BY customer_id ORDER BY order_date) as rn
FROM Delivery)


SELECT
    round(count(delivery_id) FILTER (WHERE order_date = customer_pref_delivery_date )*100.0 / count(delivery_id), 2) as immediate_percentage
FROM t
WHERE t.rn = 1
```

In [ ]:
data = [[1, 1, '2019-08-01', '2019-08-02'], [2, 2, '2019-08-02', '2019-08-02'], [3, 1, '2019-08-11', '2019-08-12'], [4, 3, '2019-08-24', '2019-08-24'], [5, 3, '2019-08-21', '2019-08-22'], [6, 2, '2019-08-11', '2019-08-13'], [7, 4, '2019-08-09', '2019-08-09']]
delivery = pd.DataFrame(data, columns=['delivery_id', 'customer_id', 'order_date', 'customer_pref_delivery_date']).astype({'delivery_id':'Int64', 'customer_id':'Int64', 'order_date':'datetime64[ns]', 'customer_pref_delivery_date':'datetime64[ns]'})

In [ ]:
def immediate_food_delivery(delivery: pd.DataFrame) -> pd.DataFrame:
    first_orders = delivery.assign(rn=delivery.groupby('customer_id').order_date.rank(method='first')).query("(rn == 1)")
    immediate_first_orders = first_orders.query("order_date == customer_pref_delivery_date")
    return pd.DataFrame([immediate_first_orders.shape[0] * 100 / first_orders.shape[0]], columns=['immediate_percentage'], index= [0]).round(2)

1193. Monthly Transactions I

https://leetcode.com/problems/monthly-transactions-i/description/


```sql
-- Write your PostgreSQL query statement below
SELECT
to_char(t.trans_date, 'YYYY-MM') as month, country,
count(*) as trans_count,
count(*) FILTER (WHERE t.state = 'approved') as approved_count,
coalesce(sum(t.amount), 0) as trans_total_amount,
coalesce(sum(t.amount) FILTER (WHERE t.state = 'approved'), 0) as approved_total_amount

FROM Transactions t

GROUP BY to_char(trans_date, 'YYYY-MM'), country
```

In [ ]:
data = [[121, 'US', 'approved', 1000, '2018-12-18'], [122, 'US', 'declined', 2000, '2018-12-19'], [123, 'US', 'approved', 2000, '2019-01-01'], [124, 'DE', 'approved', 2000, '2019-01-07']]
transactions = pd.DataFrame(data, columns=['id', 'country', 'state', 'amount', 'trans_date']).astype({'id':'Int64', 'country':'object', 'state':'object', 'amount':'Int64', 'trans_date':'datetime64[ns]'})

In [ ]:
import numpy as np
def monthly_transactions(transactions: pd.DataFrame) -> pd.DataFrame:
    return transactions.assign(month = transactions.trans_date.dt.strftime('%Y-%m'))\
            .assign(approved_total_amount = transactions.amount * np.where(transactions.state == 'approved', 1, 0))\
            .groupby(["month", "country"], dropna=False).agg({'id': 'count', 'state': lambda x: sum(np.where(x == 'approved', 1, 0)), 'amount': 'sum', 'approved_total_amount': 'sum'})\
            .rename({'id': 'trans_count', 'state':'approved_count', 'amount':'trans_total_amount'}, axis=1).reset_index()

1204. Last Person to Fit in the Bus

https://leetcode.com/problems/last-person-to-fit-in-the-bus/description/

```sql
-- Write your PostgreSQL query statement below

SELECT
person_name
FROM (
SELECT
    turn,
    person_name,
    sum(weight ) OVER (ORDER BY turn) as cumsum
FROM Queue ) sub

WHERE cumsum <= 1000
ORDER BY turn DESC
LIMIT 1
```

In [ ]:
data = [[5, 'Alice', 250, 1], [4, 'Bob', 175, 5], [3, 'Alex', 350, 2], [6, 'John Cena', 400, 3], [1, 'Winston', 500, 6], [2, 'Marie', 200, 4]]
queue = pd.DataFrame(data, columns=['person_id', 'person_name', 'weight', 'turn']).astype({'person_id':'Int64', 'person_name':'object', 'weight':'Int64', 'turn':'Int64'})

In [ ]:
def last_passenger(queue: pd.DataFrame) -> pd.DataFrame:
    person_name  = queue.sort_values(by='turn').assign(csum = queue.sort_values(by='turn').weight.cumsum()).query('csum <= 1000', engine='python').set_index('person_name').turn.idxmax()
    return pd.DataFrame([person_name], columns=['person_name'], index=[0])

1321. Restaurant Growth

https://leetcode.com/problems/restaurant-growth/description/

```sql
-- Write your PostgreSQL query statement below
WITH agged_amount AS (
    SELECT
        visited_on,
        sum(amount) as amount
    FROM Customer
    GROUP BY visited_on
),
t as (SELECT
    visited_on,
    ROUND(sum(amount) OVER (ORDER BY visited_on RANGE BETWEEN '6 days' PRECEDING AND CURRENT ROW), 2) as amount ,
    ROUND(avg(amount) OVER (ORDER BY visited_on RANGE BETWEEN '6 days' PRECEDING AND CURRENT ROW), 2) as average_amount
FROM agged_amount)

SELECT *
FROM t
WHERE visited_on >= (SELECT min(visited_on ) FROM Customer) + INTERVAL '6 days'
```

In [ ]:
data = [[1, 'Jhon', '2019-01-01', 100], [2, 'Daniel', '2019-01-02', 110], [3, 'Jade', '2019-01-03', 120], [4, 'Khaled', '2019-01-04', 130], [5, 'Winston', '2019-01-05', 110], [6, 'Elvis', '2019-01-06', 140], [7, 'Anna', '2019-01-07', 150], [8, 'Maria', '2019-01-08', 80], [9, 'Jaze', '2019-01-09', 110], [1, 'Jhon', '2019-01-10', 130], [3, 'Jade', '2019-01-10', 150]]
customer = pd.DataFrame(data, columns=['customer_id', 'name', 'visited_on', 'amount']).astype({'customer_id':'Int64', 'name':'object', 'visited_on':'datetime64[ns]', 'amount':'Int64'})

In [ ]:
def restaurant_growth(customer: pd.DataFrame) -> pd.DataFrame:
    grouped = customer.groupby('visited_on')\
                      .amount\
                      .sum()
    return grouped.rolling('7d', min_periods=7)\
                  .agg(['sum', 'mean'])\
                  .round(2)\
                  .reset_index()\
                  .rename({'sum':'amount', 'mean':'average_amount'}, axis=1)\
                  .dropna()\
                  .sort_values(by='visited_on')

1341. Movie Rating

https://leetcode.com/problems/movie-rating/description/

```sql
-- Write your PostgreSQL query statement below

WITH t as (
SELECT
    user_id,
    count(movie_id),
    rank() OVER (ORDER BY count(movie_id) DESC) as rnk
FROM MovieRating
GROUP BY user_id),

t2 as (
SELECT
    movie_id,
    avg(rating),
    rank() OVER (ORDER BY avg(rating) DESC) as rnk
FROM MovieRating
WHERE to_char(created_at, 'YYYY-MM') = '2020-02'
GROUP BY movie_id)   

(SELECT
    u.name as results
FROM t
JOIN  Users u on u.user_id = t.user_id AND t.rnk = 1
ORDER BY 1
LIMIT 1)

UNION ALL

(SELECT
    m.title as results
FROM t2
JOIN  Movies  m on m.movie_id = t2.movie_id AND t2.rnk = 1
ORDER BY 1
LIMIT 1)
```

In [ ]:
data = [[1, 'Avengers'], [2, 'Frozen 2'], [3, 'Joker']]
movies = pd.DataFrame(data, columns=['movie_id', 'title']).astype({'movie_id':'Int64', 'title':'object'})
data = [[1, 'Daniel'], [2, 'Monica'], [3, 'Maria'], [4, 'James']]
users = pd.DataFrame(data, columns=['user_id', 'name']).astype({'user_id':'Int64', 'name':'object'})
data = [[1, 1, 3, '2020-01-12'], [1, 2, 4, '2020-02-11'], [1, 3, 2, '2020-02-12'], [1, 4, 1, '2020-01-01'], [2, 1, 5, '2020-02-17'], [2, 2, 2, '2020-02-01'], [2, 3, 2, '2020-03-01'], [3, 1, 3, '2020-02-22'], [3, 2, 4, '2020-02-25']]
movie_rating = pd.DataFrame(data, columns=['movie_id', 'user_id', 'rating', 'created_at']).astype({'movie_id':'Int64', 'user_id':'Int64', 'rating':'Int64', 'created_at':'datetime64[ns]'})

In [ ]:
def movie_rating(movies: pd.DataFrame, users: pd.DataFrame, movie_rating: pd.DataFrame) -> pd.DataFrame:
    grouped = movie_rating.groupby('user_id')['movie_id'].count().reset_index(name='movie_cnt')
    grouped.assign(rnk = grouped.movie_cnt.rank(method='min', ascending=False)).query('rnk == 1')
    user_name = movie_rating.user_id.mode().to_frame().merge(users, on='user_id', how='left').sort_values(by='name').head(1)['name'].rename('results')
    movie_name = movie_rating[movie_rating.created_at.dt.strftime('%Y-%m') == '2020-02']\
                .groupby('movie_id')\
                .rating.mean()\
                .to_frame()\
                .merge(movies, left_index=True, right_on='movie_id')\
                .sort_values(by=['rating', 'title'], ascending=[False, True])\
                .head(1)['title']\
                .rename('results')
    return pd.concat([user_name, movie_name], ignore_index=True).to_frame()

1393. Capital Gain/Loss

https://leetcode.com/problems/capital-gainloss/

```sql
-- Write your PostgreSQL query statement below
SELECT stock_name,
(SUM(CASE WHEN operation = 'Sell' THEN price ELSE 0 END) - SUM (CASE WHEN operation = 'Buy' THEN price ELSE 0 END)) as capital_gain_loss
FROM Stocks GROUP BY stock_name
```

In [ ]:
data = [['Leetcode', 'Buy', 1, 1000], ['Corona Masks', 'Buy', 2, 10], ['Leetcode', 'Sell', 5, 9000], ['Handbags', 'Buy', 17, 30000], ['Corona Masks', 'Sell', 3, 1010], ['Corona Masks', 'Buy', 4, 1000], ['Corona Masks', 'Sell', 5, 500], ['Corona Masks', 'Buy', 6, 1000], ['Handbags', 'Sell', 29, 7000], ['Corona Masks', 'Sell', 10, 10000]]
stocks = pd.DataFrame(data, columns=['stock_name', 'operation', 'operation_day', 'price']).astype({'stock_name':'object', 'operation':'object', 'operation_day':'Int64', 'price':'Int64'})

In [ ]:
def capital_gainloss(stocks: pd.DataFrame) -> pd.DataFrame:
    return stocks.groupby('stock_name').apply(lambda x: np.where(x.operation == 'Sell', x.price, -x.price).sum(), include_groups=False).reset_index(name='capital_gain_loss')

1907. Count Salary Categories

https://leetcode.com/problems/count-salary-categories/description/

```sql
-- Write your PostgreSQL query statement below
WITH binned as (
SELECT
    CASE WHEN income < 20000 THEN 'Low Salary' WHEN income <= 50000 THEN 'Average Salary' ELSE 'High Salary' END AS cat
FROM Accounts)

SELECT
category,
coalesce(accounts_count , 0) as accounts_count
FROM UNNEST(ARRAY['Low Salary', 'Average Salary', 'High Salary']) AS category
LEFT JOIN (
SELECT
    cat,
    count(*) as accounts_count
FROM binned
GROUP BY cat) sub ON sub.cat = category
```

In [ ]:
data = [[3, 108939], [2, 12747], [8, 87709], [6, 91796]]
accounts = pd.DataFrame(data, columns=['account_id', 'income']).astype({'account_id':'Int64', 'income':'Int64'})

In [ ]:
def count_salary_categories(accounts: pd.DataFrame) -> pd.DataFrame:
    return pd.DataFrame({
        'category': ['Low Salary', 'Average Salary', 'High Salary'],
        'accounts_count': [
            accounts[accounts.income < 20000].shape[0],
            accounts[(accounts.income >= 20000) & (accounts.income <= 50000)].shape[0],
            accounts[accounts.income > 50000].shape[0],
        ],
    })

1934. Confirmation Rate

https://leetcode.com/problems/confirmation-rate/description/

```sql
WITH t as (
SELECT
    user_id,
    ROUND(count(action) FILTER (WHERE action = 'confirmed')*1.0 / count(action), 2) as confirmation_rate
FROM Confirmations
GROUP BY user_id
)

SELECT
s.user_id,
COALESCE(confirmation_rate, 0) as confirmation_rate
FROM Signups s
LEFT JOIN t on t.user_id = s.user_id
```

In [ ]:
data = [[3, '2020-03-21 10:16:13'], [7, '2020-01-04 13:57:59'], [2, '2020-07-29 23:09:44'], [6, '2020-12-09 10:39:37']]
signups = pd.DataFrame(data, columns=['user_id', 'time_stamp']).astype({'user_id':'Int64', 'time_stamp':'datetime64[ns]'})
data = [[3, '2021-01-06 03:30:46', 'timeout'], [3, '2021-07-14 14:00:00', 'timeout'], [7, '2021-06-12 11:57:29', 'confirmed'], [7, '2021-06-13 12:58:28', 'confirmed'], [7, '2021-06-14 13:59:27', 'confirmed'], [2, '2021-01-22 00:00:00', 'confirmed'], [2, '2021-02-28 23:59:59', 'timeout']]
confirmations = pd.DataFrame(data, columns=['user_id', 'time_stamp', 'action']).astype({'user_id':'Int64', 'time_stamp':'datetime64[ns]', 'action':'object'})

In [ ]:
def confirmation_rate(signups: pd.DataFrame, confirmations: pd.DataFrame) -> pd.DataFrame:
    grouped = confirmations.groupby('user_id').apply(lambda x: np.where(x.action  == 'confirmed', 1, 0).sum() / x.shape[0], include_groups=False).round(2).reset_index(name='confirmation_rate')
    return signups.merge(grouped, how='left', on='user_id').fillna(0)[['user_id', 'confirmation_rate']]

3220. Odd and Even Transactions

https://leetcode.com/problems/odd-and-even-transactions/description/

```sql
SELECT
    transaction_date,
    COALESCE(sum(amount) FILTER (WHERE amount % 2 = 1 ), 0) as odd_sum,
    COALESCE(sum(amount) FILTER (WHERE amount % 2 = 0 ), 0) as even_sum

FROM transactions
GROUP BY transaction_date
ORDER BY transaction_date
```

In [ ]:
data = [
    [1, 150, '2024-07-01'],
    [2, 200, '2024-07-01'],
    [3, 75, '2024-07-01'],
    [4, 300, '2024-07-02'],
    [5, 50, '2024-07-02'],
    [6, 120, '2024-07-03']
]
transactions = pd.DataFrame(data, columns=["transaction_id", "amount", "transaction_date"])
transactions["transaction_date"] = pd.to_datetime(transactions["transaction_date"])

In [ ]:
def sum_daily_odd_even(transactions: pd.DataFrame) -> pd.DataFrame:
    result = transactions.groupby('transaction_date')\
                         .agg({'amount': [lambda x: np.where(x % 2, x, 0).sum(), lambda x: np.where(x % 2, 0, x).sum()]})\
                         .rename({'<lambda_0>':'odd_sum', '<lambda_1>': 'even_sum'}, axis=1)
    result.columns = result.columns\
                           .droplevel()
    return result.reset_index()\
                 .sort_values(by='transaction_date')

3421. Find Students Who Improved

https://leetcode.com/problems/find-students-who-improved/description/

```sql
-- Write your PostgreSQL query statement below

WITH marks as (
SELECT
DISTINCT s.student_id, s.subject,
FIRST_VALUE(score) OVER (PARTITION BY (s.student_id, s.subject) ORDER BY exam_date
) as first_score,
LAST_VALUE(score) OVER (PARTITION BY (s.student_id, s.subject) ORDER BY exam_date
ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as latest_score
FROM Scores s)

SELECT * FROM marks
WHERE latest_score > first_score
ORDER BY 1, 2
```

In [ ]:
data = [
    [101, 'Math', 70, '2023-01-15'],
    [101, 'Math', 85, '2023-02-15'],
    [101, 'Physics', 65, '2023-01-15'],
    [101, 'Physics', 60, '2023-02-15'],
    [102, 'Math', 80, '2023-01-15'],
    [102, 'Math', 85, '2023-02-15'],
    [103, 'Math', 90, '2023-01-15'],
    [104, 'Physics', 75, '2023-01-15'],
    [104, 'Physics', 85, '2023-02-15']
]

# Создаем DataFrame
scores = pd.DataFrame(data, columns=["student_id", "subject", "score", "exam_date"])

# Преобразуем exam_date в datetime
scores["exam_date"] = pd.to_datetime(scores["exam_date"])

In [ ]:
def find_students_who_improved(scores: pd.DataFrame) -> pd.DataFrame:
    scores.sort_values(by=['student_id', 'subject', 'exam_date'], inplace=True)
    return scores.groupby(['student_id', 'subject'], as_index=False).agg(first_score=('score', 'first'), latest_score =('score', 'last')).query("latest_score  > first_score")

3475. DNA Pattern Recognition

https://leetcode.com/problems/dna-pattern-recognition/description/

```sql
-- Write your PostgreSQL query statement below
SELECT
sample_id, dna_sequence, species,
CASE WHEN dna_sequence like 'ATG%' THEN 1 ELSE 0 END AS has_start,
CASE WHEN dna_sequence ~ '(TAA|TAG|TGA)$' THEN 1 ELSE 0 END AS has_stop,
CASE WHEN dna_sequence like '%ATAT%' THEN 1 ELSE 0 END AS has_atat,
CASE WHEN dna_sequence ~ 'G{3,}' THEN 1 ELSE 0 END AS has_ggg
FROM Samples
ORDER BY 1
```

In [ ]:
data = [[1, 'ATGCTAGCTAGCTAA', 'Human'], [2, 'GGGTCAATCATC', 'Human'], [3, 'ATATATCGTAGCTA', 'Human'], [4, 'ATGGGGTCATCATAA', 'Mouse'], [5, 'TCAGTCAGTCAG', 'Mouse'], [6, 'ATATCGCGCTAG', 'Zebrafish'], [7, 'CGTATGCGTCGTA', 'Zebrafish']]
samples = pd.DataFrame(data, columns=['sample_id',
    'dna_sequence',
    'species'])

In [ ]:
import numpy as np

def analyze_dna_patterns(samples: pd.DataFrame) -> pd.DataFrame:
    return samples.assign(has_start = np.where(samples.dna_sequence.str.match('ATG'), 1, 0),
                          has_stop = np.where(samples.dna_sequence.str.contains('(TAA|TAG|TGA)$'), 1, 0),
                          has_atat = np.where(samples.dna_sequence.str.contains('ATAT'), 1, 0),
                          has_ggg = np.where(samples.dna_sequence.str.contains('G{3,}'), 1, 0)
                          ).sort_values(by='sample_id')

574. Winning Candidate

https://leetcode.com/problems/winning-candidate/description/

```sql
WITH top_candidate as (
SELECT
v.candidateId, count(id)
FROM Vote v
GROUP BY v.candidateId
ORDER BY 2 DESC
LIMIT 1)

SELECT c.name from Candidate c WHERE c.id = (SELECT candidateId FROM top_candidate)
```

In [ ]:
data = [[1, 'A'], [2, 'B'], [3, 'C'], [4, 'D'], [5, 'E']]
candidate = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})
data = [[1, 2], [2, 4], [3, 3], [4, 2], [5, 5]]
vote = pd.DataFrame(data, columns=['id', 'candidateId']).astype({'id':'Int64', 'candidateId':'Int64'})

In [ ]:
def winning_candidate(candidate: pd.DataFrame, vote: pd.DataFrame) -> pd.DataFrame:
    return candidate[candidate.id == vote.candidateId.value_counts().idxmax()][['name']]

578. Get Highest Answer Rate Question

https://leetcode.com/problems/get-highest-answer-rate-question/description/

```sql
WITH answer_rate as (
SELECT
    question_id ,
    dense_rank() OVER (ORDER BY sum(CASE WHEN action  = 'answer' and (answer_id IS NOT NULL) THEN 1 ELSE 0 END)*1.0 / sum(CASE WHEN action = 'show' then 1 else 0 end) DESC) as rn
FROM SurveyLog s
GROUP BY question_id)

SELECT question_id as survey_log FROM  answer_rate WHERE rn = 1
ORDER BY question_id LIMIT 1
```

In [ ]:
data = [[5, 'show', 285, None, 1, 123], [5, 'answer', 285, 124124, 1, 124], [5, 'show', 369, None, 2, 125], [5, 'skip', 369, None, 2, 126]]
survey_log = pd.DataFrame(data, columns=['id', 'action', 'question_id', 'answer_id', 'q_num', 'timestamp']).astype({'id':'Int64', 'action':'object', 'question_id':'Int64', 'answer_id':'Int64', 'q_num':'Int64', 'timestamp':'Int64'})

In [ ]:
def get_the_question(survey_log: pd.DataFrame) -> pd.DataFrame:
    if survey_log.shape[0]:
        return survey_log.groupby('question_id')\
                         .apply(lambda x: sum(~x.answer_id.isna()) / sum(x.action == 'show'), include_groups=False)\
                         .reset_index()\
                         .sort_values(by=[0, 'question_id'], ascending=[False, True])\
                         .head(1)\
                         .rename({'question_id':"survey_log"}, axis=1)[['survey_log']]
    return pd.DataFrame(columns=['survey_log'])

580. Count Student Number in Departments

https://leetcode.com/problems/count-student-number-in-departments/description/

```sql
SELECT
d.dept_name,
COALESCE(sub.student_number, 0) as student_number

FROM Department d
LEFT JOIN (

SELECT
s.dept_id,
count(student_id ) as student_number
FROM Student s
GROUP BY s.dept_id) sub

ON sub.dept_id = d.dept_id
```

In [ ]:
data = [[1, 'Jack', 'M', 1], [2, 'Jane', 'F', 1], [3, 'Mark', 'M', 2]]
student = pd.DataFrame(data, columns=['student_id', 'student_name', 'gender', 'dept_id']).astype({'student_id':'Int64', 'student_name':'object', 'gender':'object', 'dept_id':'Int64'})
data = [[1, 'Engineering'], [2, 'Science'], [3, 'Law']]
department = pd.DataFrame(data, columns=['dept_id', 'dept_name']).astype({'dept_id':'Int64', 'dept_name':'object'})

In [ ]:
def count_students(student: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    return department.merge(student.groupby('dept_id')\
                        .agg(student_number = ('student_id', 'count')), how='left', left_on='dept_id', right_index=True)\
                        .iloc[:,1:]\
                        .fillna(0)\
                        .sort_values(by=['student_number', 'dept_name'], ascending=[False, True])

612. Shortest Distance in a Plane

https://leetcode.com/problems/shortest-distance-in-a-plane/description/

```sql
SELECT
ROUND(MIN(SQRT(POWER((p1.x - p2.x), 2) + POWER((p1.y - p2.y), 2)))::decimal, 2) as shortest
FROM Point2D p1
CROSS JOIN Point2D p2
WHERE p1.x <> p2.x OR p1.y <> p2.y
```

In [ ]:
data = [[-1, -1], [0, 0], [-1, -2]]
point2_d = pd.DataFrame(data, columns=['x', 'y']).astype({'x':'Int64', 'y':'Int64'})

In [ ]:
def shortest_distance(point2_d: pd.DataFrame) -> pd.DataFrame:
    # Преобразуем данные в массивы типа float для избежания ошибок
    coords = point2_d[['x', 'y']].to_numpy().astype(float)

    # Определим функцию для вычисления минимального расстояния
    def min_distance(x, y):
        # Вычисление расстояний между точкой (x, y) и всеми остальными точками
        distances = np.sqrt((x - coords[:, 0])**2 + (y - coords[:, 1])**2)
        distances[distances == 0] = np.inf  # Исключаем саму точку (расстояние до себя)
        return np.min(distances)

    # Векторизуем функцию min_distance
    vectorized_min_distance = np.vectorize(min_distance)

    # Вычисляем минимальные расстояния для всех точек
    shortest = vectorized_min_distance(coords[:, 0], coords[:, 1]).min().round(2)

    return pd.DataFrame([shortest], columns=['shortest'], index=[0])

In [ ]:
def shortest_distance(point2_d: pd.DataFrame) -> pd.DataFrame:
    return pd.DataFrame([point2_d.apply(lambda x: min(((x.x - a)**2 + (x.y - b)**2)**0.5
                             for a, b in point2_d.values
                             if (x.x, x.y) != (a, b)), axis=1).min().round(2)], columns=['shortest'], index=[0])

614. Second Degree Follower

https://leetcode.com/problems/second-degree-follower/description/

```sql
select
followee follower,
count(follower) num
from
follow f
where
exists
(
    select 1 from
    follow f1
    where
    f1.follower=f.followee
)
group by
followee
order by 1
```

In [ ]:
data = [['Alice', 'Bob'], ['Bob', 'Cena'], ['Bob', 'Donald'], ['Donald', 'Edward']]
follow = pd.DataFrame(data, columns=['followee', 'follower']).astype({'followee':'object', 'follower':'object'})

In [ ]:
def second_degree_follower(follow: pd.DataFrame) -> pd.DataFrame:
    return follow[follow.followee.isin(follow.follower)]\
                  .groupby("followee").agg(num=('follower', 'count'))\
                  .reset_index()\
                  .rename({'followee': 'follower'}, axis=1)\
                  .sort_values(by='follower')

1077. Project Employees III

https://leetcode.com/problems/project-employees-iii/description/

```sql
SELECT
project_id,
employee_id
FROM (

SELECT
p.project_id,
e.employee_id,
dense_rank() OVER (PARTITION BY p.project_id ORDER BY e.experience_years DESC) as rnk
FROM Project p
LEFT JOIN   Employee  e ON p.employee_id = e.employee_id) sub

WHERE sub.rnk = 1
```

In [ ]:
data = [[1, 1], [1, 2], [1, 3], [2, 1], [2, 4]]
project = pd.DataFrame(data, columns=['project_id', 'employee_id']).astype({'project_id':'Int64', 'employee_id':'Int64'})
data = [[1, 'Khaled', 3], [2, 'Ali', 2], [3, 'John', 3], [4, 'Doe', 2]]
employee = pd.DataFrame(data, columns=['employee_id', 'name', 'experience_years']).astype({'employee_id':'Int64', 'name':'object', 'experience_years':'Int64'})

In [ ]:
def project_employees(project: pd.DataFrame, employee: pd.DataFrame) -> pd.DataFrame:
    merged = project.merge(employee, how='left', on='employee_id')
    return merged.assign(rnk= merged.groupby("project_id").experience_years.rank(method='dense', ascending=False)).query("rnk == 1", engine='python').iloc[:,:2]

1107. New Users Daily Count

https://leetcode.com/problems/new-users-daily-count/description/

```sql
SELECT
activity_date as login_date,
count(user_id) as user_count

FROM (
SELECT
user_id, activity_date,
row_number() OVER (PARTITION BY user_id ORDER BY activity_date) as rn
FROM Traffic
WHERE activity = 'login' ) sub
WHERE rn = 1 AND activity_date >= '2019-06-30'::DATE - INTERVAL '90 DAYS'
GROUP BY activity_date
```

In [ ]:
data = [[1, 'login', '2019-05-01'], [1, 'homepage', '2019-05-01'], [1, 'logout', '2019-05-01'], [2, 'login', '2019-06-21'], [2, 'logout', '2019-06-21'], [3, 'login', '2019-01-01'], [3, 'jobs', '2019-01-01'], [3, 'logout', '2019-01-01'], [4, 'login', '2019-06-21'], [4, 'groups', '2019-06-21'], [4, 'logout', '2019-06-21'], [5, 'login', '2019-03-01'], [5, 'logout', '2019-03-01'], [5, 'login', '2019-06-21'], [5, 'logout', '2019-06-21']]
traffic = pd.DataFrame(data, columns=['user_id', 'activity', 'activity_date']).astype({'user_id':'Int64', 'activity':'object', 'activity_date':'datetime64[ns]'})

In [ ]:
def new_users_daily_count(traffic: pd.DataFrame) -> pd.DataFrame:
    traffic_filtered = traffic[traffic.activity == 'login']
    return traffic_filtered.assign(rn=traffic_filtered.groupby('user_id')\
                        .activity_date.rank(method='first'))\
                .query(f"(rn == 1) & (activity_date >= '{pd.to_datetime('2019-06-30') - pd.DateOffset(days=90)}')")\
                .groupby('activity_date')\
                .agg(user_count=('user_id', 'count'))\
                .reset_index()\
                .rename({'activity_date': 'login_date'}, axis=1)

1112. Highest Grade For Each Student

https://leetcode.com/problems/highest-grade-for-each-student/description/

```sql
SELECT
    student_id,
    course_id,
    grade
FROM (
SELECT
    *,
    dense_rank() OVER (PARTITION BY student_id ORDER BY grade DESC, course_id) as rnk
FROM Enrollments) sub
WHERE sub.rnk = 1
ORDER BY 1
```

In [ ]:
data = [[2, 2, 95], [2, 3, 95], [1, 1, 90], [1, 2, 99], [3, 1, 80], [3, 2, 75], [3, 3, 82]]
enrollments = pd.DataFrame(data, columns=['student_id', 'course_id', 'grade']).astype({'student_id':'Int64', 'course_id':'Int64', 'grade':'Int64'})

In [ ]:
def highest_grade(enrollments: pd.DataFrame) -> pd.DataFrame:
    return enrollments.assign(rnk=enrollments.groupby('student_id')\
                      .grade.rank(method='dense', ascending=False))\
                      .query("rnk == 1", engine='python')\
                      .groupby('student_id')\
                      .agg({'course_id': 'min', 'grade': 'max'})\
                      .sort_index()\
                      .reset_index()

1126. Active Businesses

https://leetcode.com/problems/active-businesses/description/

```sql
SELECT business_id FROM (
SELECT
*,
CASE WHEN occurrences > (avg(occurrences) OVER (PARTITION BY event_type)) THEN 1 ELSE 0 END as greater_avg
FROM Events) sub
GROUP BY business_id
HAVING sum(greater_avg) > 1
```

In [ ]:
data = [[1, 'reviews', 7], [3, 'reviews', 3], [1, 'ads', 11], [2, 'ads', 7], [3, 'ads', 6], [1, 'page views', 3], [2, 'page views', 12]]
events = pd.DataFrame(data, columns=['business_id', 'event_type', 'occurrences']).astype({'business_id':'Int64', 'event_type':'object', 'occurrences':'Int64'})

In [ ]:
def active_businesses(events: pd.DataFrame) -> pd.DataFrame:
    return events.assign(greater_avg = events.event_type.map(events.groupby('event_type')['occurrences'].mean()))\
      .query("occurrences > greater_avg", engine='python')\
      .business_id\
      .value_counts()\
      .where(lambda x: x > 1)\
      .dropna()\
      .reset_index()\
      .iloc[:,:1]

1132. Reported Posts II

https://leetcode.com/problems/reported-posts-ii/description/

```sql
WITH reported_removed as (
SELECT
action_date,
count(DISTINCT a.post_id) as cnt
FROM Actions a
INNER JOIN Removals r ON r.post_id  = a.post_id  --AND r.remove_date >= a.action_date
WHERE extra = 'spam'
GROUP BY action_date ),

reported as (
SELECT
action_date,
count(DISTINCT a.post_id) as cnt
FROM Actions a
WHERE extra = 'spam'
GROUP BY action_date
)

SELECT
round(AVG(COALESCE(rr.cnt, 0) * 100.0 / r.cnt), 2) as average_daily_percent
FROM
reported r
LEFT JOIN reported_removed rr on r.action_date  = rr.action_date
```

In [ ]:
data = [[1, 1, '2019-07-01', 'view', None], [1, 1, '2019-07-01', 'like', None], [1, 1, '2019-07-01', 'share', None], [2, 2, '2019-07-04', 'view', None], [2, 2, '2019-07-04', 'report', 'spam'], [3, 4, '2019-07-04', 'view', None], [3, 4, '2019-07-04', 'report', 'spam'], [4, 3, '2019-07-02', 'view', None], [4, 3, '2019-07-02', 'report', 'spam'], [5, 2, '2019-07-03', 'view', None], [5, 2, '2019-07-03', 'report', 'racism'], [5, 5, '2019-07-03', 'view', None], [5, 5, '2019-07-03', 'report', 'racism']]
actions = pd.DataFrame(data, columns=['user_id', 'post_id', 'action_date', 'action', 'extra']).astype({'user_id':'Int64', 'post_id':'Int64', 'action_date':'datetime64[ns]', 'action':'object', 'extra':'object'})
data = [[2, '2019-07-20'], [3, '2019-07-18']]
removals = pd.DataFrame(data, columns=['post_id', 'remove_date']).astype({'post_id':'Int64', 'remove_date':'datetime64[ns]'})

In [ ]:
def reported_posts(actions: pd.DataFrame, removals: pd.DataFrame) -> pd.DataFrame:
    reported_remove = actions[actions.extra=='spam'].merge(removals, on='post_id', how='inner').groupby('action_date').agg(reported_remove_count=('post_id', 'nunique'))
    result = actions[actions.extra=='spam'].groupby('action_date').agg(reported_count=('post_id', 'nunique')).merge(reported_remove, how='left', left_index=True, right_index=True)
    result = result.reported_remove_count.div(result.reported_count).fillna(0).mul(100).mean().round(2)
    return pd.DataFrame([result], columns=['average_daily_percent'], index=[0])

1149. Article Views II

https://leetcode.com/problems/article-views-ii/description/

```sql
SELECT
DISTINCT viewer_id as id
FROM Views v
GROUP BY viewer_id, view_date
HAVING count(DISTINCT article_id) > 1
ORDER BY 1
```

In [ ]:
data = [[1, 3, 5, '2019-08-01'], [3, 4, 5, '2019-08-01'], [1, 3, 6, '2019-08-02'], [2, 7, 7, '2019-08-01'], [2, 7, 6, '2019-08-02'], [4, 7, 1, '2019-07-22'], [3, 4, 4, '2019-07-21'], [3, 4, 4, '2019-07-21']]
views = pd.DataFrame(data, columns=['article_id', 'author_id', 'viewer_id', 'view_date']).astype({'article_id':'Int64', 'author_id':'Int64', 'viewer_id':'Int64', 'view_date':'datetime64[ns]'})

In [ ]:
def article_views(views: pd.DataFrame) -> pd.DataFrame:
    return views.groupby(['viewer_id', 'view_date'])\
                .agg(view_cnt = ('article_id', 'nunique'))\
                .where(lambda x: x > 1)\
                .dropna()\
                .reset_index()[['viewer_id']]\
                .rename({'viewer_id': 'id'}, axis=1)\
                .drop_duplicates()\
                .sort_values(by='id')

1205. Monthly Transactions II

https://leetcode.com/problems/monthly-transactions-ii/description/

```sql

```

In [ ]:
data = [[101, 'US', 'approved', 1000, '2019-05-18'], [102, 'US', 'declined', 2000, '2019-05-19'], [103, 'US', 'approved', 3000, '2019-06-10'], [104, 'US', 'declined', 4000, '2019-06-13'], [105, 'US', 'approved', 5000, '2019-06-15']]
transactions = pd.DataFrame(data, columns=['id', 'country', 'state', 'amount', 'trans_date']).astype({'id':'Int64', 'country':'object', 'state':'object', 'amount':'Int64', 'trans_date':'datetime64[ns]'})
data = [[102, '2019-05-29'], [101, '2019-06-30'], [105, '2019-09-18']]
chargebacks = pd.DataFrame(data, columns=['trans_id', 'trans_date']).astype({'trans_id':'Int64', 'trans_date':'datetime64[ns]'})

In [ ]:
def monthly_transactions(transactions: pd.DataFrame, chargebacks: pd.DataFrame) -> pd.DataFrame:
    chargebacks_full = chargebacks.merge(transactions, how='left', left_on='trans_id', right_on='id', suffixes=['_c', '_t'])\
                    .assign(month = pd.to_datetime(chargebacks['trans_date']).dt.strftime('%Y-%m'))\
                    .assign(state='chrgeback')\
                    [['month','id','country', 'amount', 'state']]
    transactions_full = transactions.query("state == 'approved'")\
                        .assign(month = pd.to_datetime(transactions['trans_date'])\
                        .dt.strftime('%Y-%m'))\
                        [['id', 'country', 'state', 'amount', 'month']]
    pivot = pd.concat([chargebacks_full, transactions_full], axis=0).pivot_table(index=['month', 'country'], columns=['state'], values=['amount'], aggfunc=['sum', 'count']).fillna(0).reset_index()
    pivot.columns = ['month', 'country', 'approved_amount', 'chargeback_amount', 'approved_count', 'chargeback_count']
    return pivot.reindex(columns=['month', 'country', 'approved_count', 'approved_amount', 'chargeback_count', 'chargeback_amount' ])

1212. Team Scores in Football Tournament

https://leetcode.com/problems/team-scores-in-football-tournament/description/

```sql
with all_games as (
SELECT
host_team, host_goals, guest_goals
FROM Matches
UNION ALL
SELECT guest_team, guest_goals, host_goals
FROM Matches)


SELECT
team_id, team_name,
sum(CASE WHEN host_goals > guest_goals  THEN 3 WHEN host_goals = guest_goals THEN 1 ELSE 0 END) AS num_points
FROM all_games a
RIGHT JOIN Teams t on a.host_team = t.team_id
GROUP BY team_id, team_name
ORDER BY num_points DESC, team_id

```

In [ ]:
data = [[10, 'Leetcode FC'], [20, 'NewYork FC'], [30, 'Atlanta FC'], [40, 'Chicago FC'], [50, 'Toronto FC']]
teams = pd.DataFrame(data, columns=['team_id', 'team_name']).astype({'team_id':'Int64', 'team_name':'object'})
data = [[1, 10, 20, 3, 0], [2, 30, 10, 2, 2], [3, 10, 50, 5, 1], [4, 20, 30, 1, 0], [5, 50, 30, 1, 0]]
matches = pd.DataFrame(data, columns=['match_id', 'host_team', 'guest_team', 'host_goals', 'guest_goals']).astype({'match_id':'Int64', 'host_team':'Int64', 'guest_team':'Int64', 'host_goals':'Int64', 'guest_goals':'Int64'})

In [ ]:
def team_scores(teams: pd.DataFrame, matches: pd.DataFrame) -> pd.DataFrame:
    points = matches.assign(host_points = np.where(matches.host_goals > matches.guest_goals, 3, np.where(matches.host_goals == matches.guest_goals, 1, 0)),
    guest_points = np.where(matches.host_goals < matches.guest_goals, 3, np.where(matches.host_goals == matches.guest_goals, 1, 0)))[['host_team','guest_team', 'host_points',	'guest_points']]
    teams_pints = pd.concat([pd.melt(points, value_vars=['host_team',	'guest_team'], value_name='team'), pd.melt(points, value_vars=['host_points','guest_points'], value_name='num_points')], axis=1)\
    .groupby('team').num_points.sum()
    return teams.merge(teams_pints, left_on='team_id', right_index=True, how='left').fillna(0).sort_values(by=['num_points', 'team_id'], ascending=[False, True])

1270. All People Report to the Given Manager

https://leetcode.com/problems/all-people-report-to-the-given-manager/description/


```sql
SELECT employee_id FROM (
SELECT
 distinct unnest(array_agg (l1.employee_id) || array_agg(l2.employee_id) || array_agg(l3.employee_id)) as employee_id
FROM Employees l1
LEFT JOIN Employees l2 ON l2.manager_id = l1.employee_id
LEFT JOIN Employees l3 ON l3.manager_id = l2.employee_id
WHERE l1.manager_id = 1) sub
WHERE employee_id > 1
```

In [ ]:
data = [[1, 'Boss', 1], [3, 'Alice', 3], [2, 'Bob', 1], [4, 'Daniel', 2], [7, 'Luis', 4], [8, 'John', 3], [9, 'Angela', 8], [77, 'Robert', 1]]
employees = pd.DataFrame(data, columns=['employee_id', 'employee_name', 'manager_id']).astype({'employee_id':'Int64', 'employee_name':'object', 'manager_id':'Int64'})

In [ ]:
def find_reporting_people(employees: pd.DataFrame) -> pd.DataFrame:
    employees = employees[['employee_id', 'manager_id']]
    emps = employees[employees.manager_id == 1]\
         .merge(employees, how='left', left_on='employee_id', right_on='manager_id', suffixes=('_l1', '_r1'))\
         .merge(employees, how='left', left_on='employee_id_l1', right_on='manager_id', suffixes=('_l1', '_r2'))\
         .merge(employees, how='left', left_on='employee_id', right_on='manager_id', suffixes=('_l2', '_r2'))\
         .merge(employees, how='left', left_on='employee_id_l2', right_on='manager_id', suffixes=('_l2', '_r2'))['employee_id'].where(lambda x: x.gt(1)).dropna().unique()
    return pd.DataFrame(emps, columns=['employee_id'], index=list(range(len(emps)))) if any(emps) else pd.DataFrame(columns=['employee_id'])

In [ ]:
def find_reporting_people(employees: pd.DataFrame) -> pd.DataFrame:

    new_reports =  all_reports = set([1])

    for _ in range(3):
        new_reports = set(
            employees[employees['manager_id'].isin(new_reports)]['employee_id']
        )
        if not new_reports:
            break

        all_reports.update(new_reports)
    return pd.DataFrame({'employee_id': list(all_reports - {1})})

1285. Find the Start and End Number of Continuous Ranges

https://leetcode.com/problems/find-the-start-and-end-number-of-continuous-ranges/description/

```sql
SELECT MIN(log_id ) AS start_id , MAX(log_id ) AS end_id
FROM (

SELECT
    log_id,
    log_id-row_number() OVER (ORDER BY log_id ASC) AS RN
FROM Logs)
GROUP BY RN
ORDER BY 1
```

In [ ]:
data = [[1], [2], [3], [7], [8], [10]]
logs = pd.DataFrame(data, columns=['log_id']).astype({'log_id':'Int64'})

In [ ]:
def find_continuous_ranges(logs: pd.DataFrame) -> pd.DataFrame:
    return logs.assign(rnk_diff=logs.log_id - logs.log_id.rank(method='first', ascending=True))\
            .groupby("rnk_diff")\
            .agg(start_id  = ("log_id",'min'), end_id = ("log_id", 'max'))\
            .sort_values(by='start_id')

1308. Running Total for Different Genders

https://leetcode.com/problems/running-total-for-different-genders/description/

```sql
SELECT
gender, day,
sum(score_points) OVER (PARTITION BY gender ORDER BY day) as total

FROM Scores s
ORDER BY 1, 2
```

In [ ]:
data = [['Aron', 'F', '2020-01-01', 17], ['Alice', 'F', '2020-01-07', 23], ['Bajrang', 'M', '2020-01-07', 7], ['Khali', 'M', '2019-12-25', 11], ['Slaman', 'M', '2019-12-30', 13], ['Joe', 'M', '2019-12-31', 3], ['Jose', 'M', '2019-12-18', 2], ['Priya', 'F', '2019-12-31', 23], ['Priyanka', 'F', '2019-12-30', 17]]
scores = pd.DataFrame(data, columns=['player_name', 'gender', 'day', 'score_points']).astype({'player_name':'object', 'gender':'object', 'day':'datetime64[ns]', 'score_points':'Int64'})

In [ ]:
def running_total(scores: pd.DataFrame) -> pd.DataFrame:
    scores.sort_values(by=['gender', 'day'], inplace=True)
    return scores.assign(total = scores.groupby('gender').score_points.cumsum())[['gender', 'day', 'total']]

1355. Activity Participants

https://leetcode.com/problems/activity-participants/description/

```sql
SELECT
sub.name as activity FROM (
SELECT
a.name,
count(f.activity) as act_cnt,
dense_rank() OVER (ORDER BY count(f.activity) ) as rnk_asc,
dense_rank() OVER (ORDER BY count(f.activity) DESC) as rnk_desc
FROM Activities a
LEFT JOIN Friends f on f.activity = a.name
GROUP BY a.name) sub
WHERE rnk_asc > 1 AND rnk_desc > 1
```

In [ ]:
data = [[1, 'Jonathan D.', 'Eating'], [2, 'Jade W.', 'Singing'], [3, 'Victor J.', 'Singing'], [4, 'Elvis Q.', 'Eating'], [5, 'Daniel A.', 'Eating'], [6, 'Bob B.', 'Horse Riding']]
friends = pd.DataFrame(data, columns=['id', 'name', 'activity']).astype({'id':'Int64', 'name':'object', 'activity':'object'})
data = [[1, 'Eating'], [2, 'Singing'], [3, 'Horse Riding']]
activities = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})

In [ ]:
def activity_participants(friends: pd.DataFrame, activities: pd.DataFrame) -> pd.DataFrame:
    agged_df = activities.merge(friends, how='left', left_on='name', right_on='activity')\
                .groupby('name_x')\
                .agg(act_cnt = ('activity', 'count')).reset_index()\

    return agged_df.assign(rnk_asc = agged_df.act_cnt.rank(method='dense', ascending=True),
                    rnk_desc = agged_df.act_cnt.rank(method='dense', ascending=False))\
                                       .query("rnk_asc > 1 & rnk_desc > 1")[['name_x']]\
                                       .rename({"name_x": "activity"}, axis=1)

1364. Number of Trusted Contacts of a Customer

https://leetcode.com/problems/number-of-trusted-contacts-of-a-customer/description/

```sql


```

1141. User Activity for the Past 30 Days I

https://leetcode.com/problems/user-activity-for-the-past-30-days-i/description/

```sql
SELECT
activity_date AS day,
count(DISTINCT user_id) AS active_users
FROM Activity
WHERE activity_date BETWEEN '2019-07-27'::DATE - INTERVAL '29 DAYS' AND '2019-07-27'
GROUP BY activity_date
ORDER BY 1
```

In [ ]:
data = [[1, 1, '2019-07-20', 'open_session'], [1, 1, '2019-07-20', 'scroll_down'], [1, 1, '2019-07-20', 'end_session'], [2, 4, '2019-07-20', 'open_session'], [2, 4, '2019-07-21', 'send_message'], [2, 4, '2019-07-21', 'end_session'], [3, 2, '2019-07-21', 'open_session'], [3, 2, '2019-07-21', 'send_message'], [3, 2, '2019-07-21', 'end_session'], [4, 3, '2019-06-25', 'open_session'], [4, 3, '2019-06-25', 'end_session']]
activity = pd.DataFrame(data, columns=['user_id', 'session_id', 'activity_date', 'activity_type']).astype({'user_id':'Int64', 'session_id':'Int64', 'activity_date':'datetime64[ns]', 'activity_type':'object'})

In [ ]:
def user_activity(activity: pd.DataFrame) -> pd.DataFrame:
    return activity[activity.activity_date.between(pd.to_datetime('2019-07-27', format='%Y-%m-%d') - pd.DateOffset(days=29),\
                                        pd.to_datetime('2019-07-27', format='%Y-%m-%d'))]\
                                        .groupby('activity_date')\
                                        .agg(active_users  = ('user_id', 'nunique'))\
                                        .reset_index().rename({'activity_date': 'day'}, axis=1)\
                                        .sort_values(by='day')

1398. Customers Who Bought Products A and B but Not C

https://leetcode.com/problems/customers-who-bought-products-a-and-b-but-not-c/

```sql
SELECT
DISTINCT c.customer_id,
c.customer_name
FROM Orders o
LEFT JOIN Customers c ON c.customer_id  = o.customer_id
WHERE
EXISTS (SELECT 1 FROM Orders o1 WHERE  o1.customer_id = o.customer_id AND  o1.product_name  = 'A' ) AND
EXISTS (SELECT 1 FROM Orders o2 WHERE  o2.customer_id = o.customer_id AND  o2.product_name  = 'B' ) AND
NOT EXISTS (SELECT 1 FROM Orders o3 WHERE  o3.customer_id = o.customer_id AND  o3.product_name  = 'C' )
ORDER BY 1
```

In [ ]:
data = [[1, 'Daniel'], [2, 'Diana'], [3, 'Elizabeth'], [4, 'Jhon']]
customers = pd.DataFrame(data, columns=['customer_id', 'customer_name']).astype({'customer_id':'Int64', 'customer_name':'object'})
data = [[10, 1, 'A'], [20, 1, 'B'], [30, 1, 'D'], [40, 1, 'C'], [50, 2, 'A'], [60, 3, 'A'], [70, 3, 'B'], [80, 3, 'D'], [90, 4, 'C']]
orders = pd.DataFrame(data, columns=['order_id', 'customer_id', 'product_name']).astype({'order_id':'Int64', 'customer_id':'Int64', 'product_name':'object'})

In [ ]:
def find_customers(customers: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    filtered_customers = (
    orders.groupby('customer_id')['product_name']
    .apply(set)
    .loc[lambda x: x.apply(lambda s: {'A', 'B'}.issubset(s) and 'C' not in s)]
)
    return filtered_customers.reset_index().merge(customers, how='left', left_on='customer_id', right_on='customer_id')[['customer_id', 'customer_name']].sort_values(by='customer_id')

In [ ]:
def find_customers(customers: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    grouped = orders.groupby('customer_id')['product_name']\
          .apply(set)\
          .reset_index()
    return grouped[grouped.product_name.apply(lambda s: {'A', 'B'}.issubset(s) and 'C' not in s)]\
        .merge(customers, how='left', left_on='customer_id', right_on='customer_id')\
         [['customer_id', 'customer_name']]\
         .sort_values(by='customer_id')

1440. Evaluate Boolean Expression

https://leetcode.com/problems/evaluate-boolean-expression/description/

```sql
SELECT
e.*,
CASE
WHEN e.operator = '=' THEN (l.value = r.value)::TEXT
WHEN e.operator = '>' THEN (l.value > r.value)::TEXT
WHEN e.operator = '<' THEN (l.value < r.value)::TEXT
END AS value
FROM Expressions e
LEFT JOIN Variables l ON l.name = e.left_operand
LEFT JOIN Variables r ON r.name = e.right_operand
```

In [ ]:
data = [['x', 66], ['y', 77]]
variables = pd.DataFrame(data, columns=['name', 'value']).astype({'name':'object', 'value':'Int64'})
data = [['x', '>', 'y'], ['x', '<', 'y'], ['x', '=', 'y'], ['y', '>', 'x'], ['y', '<', 'x'], ['x', '=', 'x']]
expressions = pd.DataFrame(data, columns=['left_operand', 'operator', 'right_operand']).astype({'left_operand':'object', 'operator':'object', 'right_operand':'object'})

In [ ]:
import numpy as np
import pandas as pd


def eval_expression(variables: pd.DataFrame, expressions: pd.DataFrame) -> pd.DataFrame:
    if expressions.shape[0]:
        d = {k: v for k, v in zip(variables.name, variables.value)}
        expressions['l'] = expressions.left_operand.map(d)
        expressions['r'] = expressions.right_operand.map(d)
        def caluclate(l, r, op):
            if op == '>':
                return str(l > r).lower()
            elif op == '<':
                return str(l < r).lower()
            return str(l == r).lower()
        expressions['value'] = np.vectorize(caluclate)(expressions['l'], expressions['r'], expressions['operator'])
        return expressions.drop(columns=['l', 'r'])
    return pd.DataFrame(columns=list(expressions.columns) + ['value'])

1445. Apples & Oranges

https://leetcode.com/problems/apples-oranges/description/

```sql
SELECT
sale_date,
sum(case when fruit = 'apples' then sold_num else 0 end) -
sum(case when fruit = 'oranges' then sold_num else 0 end) as diff
FROM Sales
GROUP BY sale_date
order by 1
```

In [ ]:
data = [['2020-05-01', 'apples', 10], ['2020-05-01', 'oranges', 8], ['2020-05-02', 'apples', 15], ['2020-05-02', 'oranges', 15], ['2020-05-03', 'apples', 20], ['2020-05-03', 'oranges', 0], ['2020-05-04', 'apples', 15], ['2020-05-04', 'oranges', 16]]
sales = pd.DataFrame(data, columns=['sale_date', 'fruit', 'sold_num']).astype({'sale_date':'datetime64[ns]', 'fruit':'object', 'sold_num':'Int64'})

In [ ]:
def apples_oranges(sales: pd.DataFrame) -> pd.DataFrame:
    return sales.pivot_table(index='sale_date', columns=['fruit'], values=['sold_num']).diff(-1, axis=1)[('sold_num',  'apples')].reset_index(name='diff')

1454. Active Users

https://leetcode.com/problems/active-users/description/

```sql
WITH t AS (
    SELECT id, login_date,
    dense_rank() OVER (PARTITION BY id ORDER BY login_date ) AS rnk
    FROM logins
)
SELECT DISTINCT c.id, a.name FROM t c
JOIN accounts a ON c.id = a.id
GROUP BY c.id, a.name, c.login_date -  rnk::INT
HAVING count(DISTINCT c.login_date) >=5
```

In [ ]:
data = [[1, 'Winston'], [7, 'Jonathan']]
accounts = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})
data = [[7, '2020-05-30'], [1, '2020-05-30'], [7, '2020-05-31'], [7, '2020-06-01'], [7, '2020-06-02'], [7, '2020-06-02'], [7, '2020-06-03'], [1, '2020-06-07'], [7, '2020-06-10']]
logins = pd.DataFrame(data, columns=['id', 'login_date']).astype({'id':'Int64', 'login_date':'datetime64[ns]'})

In [ ]:
def active_users(accounts: pd.DataFrame, logins: pd.DataFrame) -> pd.DataFrame:
    logins = logins.drop_duplicates()
    logins = logins.assign(rnk=logins.groupby(['id']).login_date.rank(method='first'))
    return logins.assign(dif=logins.login_date - logins.rnk.apply(lambda x: pd.to_timedelta(x, unit='D')))\
        .groupby(['id', 'dif'])\
        .agg(cnt=('login_date', 'count')).reset_index().query("cnt > 4").merge(accounts, how='left', on='id')[['id', 'name']].drop_duplicates()

1532. The Most Recent Three Orders

https://leetcode.com/problems/the-most-recent-three-orders/description/

```sql
SELECT c.name as customer_name , sub.customer_id, sub.order_id, sub.order_date FROM (
SELECT
customer_id, order_id, order_date,
row_number() OVER (PARTITION BY customer_id ORDER BY order_date DESC) as rn
FROM Orders o) sub
RIGHT JOIN Customers c USING(customer_id)
WHERE rn < 4
ORDER BY 1, 2, 4 DESC
```

In [ ]:
data = [[1, 'Winston'], [2, 'Jonathan'], [3, 'Annabelle'], [4, 'Marwan'], [5, 'Khaled']]
customers = pd.DataFrame(data, columns=['customer_id', 'name']).astype({'customer_id':'Int64', 'name':'object'})
data = [[1, '2020-07-31', 1, 30], [2, '2020-7-30', 2, 40], [3, '2020-07-31', 3, 70], [4, '2020-07-29', 4, 100], [5, '2020-06-10', 1, 1010], [6, '2020-08-01', 2, 102], [7, '2020-08-01', 3, 111], [8, '2020-08-03', 1, 99], [9, '2020-08-07', 2, 32], [10, '2020-07-15', 1, 2]]
orders = pd.DataFrame(data, columns=['order_id', 'order_date', 'customer_id', 'cost']).astype({'order_id':'Int64', 'order_date':'datetime64[ns]', 'customer_id':'Int64', 'cost':'Int64'})

In [ ]:
def recent_three_orders(customers: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    return orders.assign(rn = orders.groupby('customer_id')\
              .order_date.rank(method='first', ascending=False))\
              .query("rn < 4")\
              .merge(customers, how='left', on='customer_id')\
              [['name', 'customer_id', 'order_id', 'order_date']]\
              .rename({'name': 'customer_name'}, axis=1).sort_values(by=['customer_name','customer_id', 'order_date'], ascending=[True, True, False])

1468. Calculate Salaries

https://leetcode.com/problems/calculate-salaries/

```sql
WITH taxes as (
SELECT
company_id,
CASE WHEN max(salary) < 1000 THEN 1
WHEN 1000 <= max(salary) AND max(salary) <= 10000 THEN 0.76
ELSE 0.51 END as coef
FROM Salaries
GROUP BY company_id)

SELECT
company_id, employee_id, employee_name,
ROUND(salary  * coef, 0) as salary
FROM Salaries
INNER JOIN taxes USING(company_id )
```

In [ ]:
data = [[1, 1, 'Tony', 2000], [1, 2, 'Pronub', 21300], [1, 3, 'Tyrrox', 10800], [2, 1, 'Pam', 300], [2, 7, 'Bassem', 450], [2, 9, 'Hermione', 700], [3, 7, 'Bocaben', 100], [3, 2, 'Ognjen', 2200], [3, 13, 'Nyancat', 3300], [3, 15, 'Morninngcat', 7777]]
salaries = pd.DataFrame(data, columns=['company_id', 'employee_id', 'employee_name', 'salary']).astype({'company_id':'Int64', 'employee_id':'Int64', 'employee_name':'object', 'salary':'Int64'})

In [ ]:
import numpy as np

In [ ]:
def calculate_salaries(salaries: pd.DataFrame) -> pd.DataFrame:
    taxes_coefs = salaries.groupby('company_id').salary.max().apply(lambda x: np.where(x < 1000, 1, np.where(x <= 10000, 0.76, 0.51)))
    salaries['salary'] = (salaries['salary'] * salaries.company_id.map(taxes_coefs)).round().astype('int')
    return salaries

1549. The Most Recent Orders for Each Product

https://leetcode.com/problems/the-most-recent-orders-for-each-product/

```sql
SELECT p.product_name, sub.product_id, sub.order_id, sub.order_date FROM (
SELECT
product_id,order_id, order_date,
rank() OVER (PARTITION BY product_id ORDER BY order_date DESC) as rn
FROM Orders o) sub
INNER JOIN Products p USING(product_id )
WHERE rn = 1
ORDER BY 1, 2, 3
```

In [ ]:
data = [[1, 'Winston'], [2, 'Jonathan'], [3, 'Annabelle'], [4, 'Marwan'], [5, 'Khaled']]
customers = pd.DataFrame(data, columns=['customer_id', 'name']).astype({'customer_id':'Int64', 'name':'object'})
data = [[1, '2020-07-31', 1, 1], [2, '2020-7-30', 2, 2], [3, '2020-08-29', 3, 3], [4, '2020-07-29', 4, 1], [5, '2020-06-10', 1, 2], [6, '2020-08-01', 2, 1], [7, '2020-08-01', 3, 1], [8, '2020-08-03', 1, 2], [9, '2020-08-07', 2, 3], [10, '2020-07-15', 1, 2]]
orders = pd.DataFrame(data, columns=['order_id', 'order_date', 'customer_id', 'product_id']).astype({'order_id':'Int64', 'order_date':'datetime64[ns]', 'customer_id':'Int64', 'product_id':'Int64'})
data = [[1, 'keyboard', 120], [2, 'mouse', 80], [3, 'screen', 600], [4, 'hard disk', 450]]
products = pd.DataFrame(data, columns=['product_id', 'product_name', 'price']).astype({'product_id':'Int64', 'product_name':'object', 'price':'Int64'})

In [ ]:
orders.assign(rnk=orders.groupby('product_id').order_date.rank(method='min', ascending=False))\
      .query('rnk == 1')\
      .merge(products, how='inner', on='product_id')[['product_name', 'product_id', 'order_date', 'order_id']]\
      .sort_values(by=['product_name', 'product_id', 'order_id'])

,product_name,product_id,order_date,order_id
1,keyboard,1,2020-08-01,6
2,keyboard,1,2020-08-01,7
3,mouse,2,2020-08-03,8
0,screen,3,2020-08-29,3


1501. Countries You Can Safely Invest In

https://leetcode.com/problems/countries-you-can-safely-invest-in/description/

```sql
WITH all_calls AS (
(SELECT
caller_id as id, duration,
SUBSTRING(p.phone_number, 1, 3) as country_code
FROM Calls c
LEFT JOIN Person p on p.id = c.caller_id)

UNION ALL

(SELECT
callee_id as id,
duration,
SUBSTRING(p.phone_number, 1, 3) as country_code
FROM Calls c
LEFT JOIN Person p on p.id = c.callee_id))

SELECT
c.name as country
FROM all_calls a
LEFT JOIN Country c USING(country_code)
GROUP BY a.country_code, c.name
HAVING avg(a.duration) > (SELECT avg(duration) FROM Calls)
```

In [2]:
data = [[3, 'Jonathan', '051-1234567'], [12, 'Elvis', '051-7654321'], [1, 'Moncef', '212-1234567'], [2, 'Maroua', '212-6523651'], [7, 'Meir', '972-1234567'], [9, 'Rachel', '972-0011100']]
person = pd.DataFrame(data, columns=['id', 'name', 'phone_number']).astype({'id':'Int64', 'name':'object', 'phone_number':'object'})
data = [['Peru', '051'], ['Israel', '972'], ['Morocco', '212'], ['Germany', '049'], ['Ethiopia', '251']]
country = pd.DataFrame(data, columns=['name', 'country_code']).astype({'name':'object', 'country_code':'object'})
data = [[1, 9, 33], [2, 9, 4], [1, 2, 59], [3, 12, 102], [3, 12, 330], [12, 3, 5], [7, 9, 13], [7, 1, 3], [9, 7, 1], [1, 7, 7]]
calls = pd.DataFrame(data, columns=['caller_id', 'callee_id', 'duration']).astype({'caller_id':'Int64', 'callee_id':'Int64', 'duration':'Int64'})

In [36]:
def find_safe_countries(person: pd.DataFrame, country: pd.DataFrame, calls: pd.DataFrame) -> pd.DataFrame:
    df_union = pd.concat([calls[['caller_id', 'duration']].rename({'caller_id': 'id'}, axis=1), calls[['callee_id', 'duration']].rename({'callee_id': 'id'}, axis=1)])
    union_merged = df_union.merge(person, how='left', on='id')
    return union_merged.assign(country_code = union_merged['phone_number']\
                        .apply(lambda x: x[:3]))\
                        .groupby('country_code')\
                        .duration.mean()\
                        .where(lambda x: x > calls.duration.mean()).dropna().reset_index().merge(country, how='left', on='country_code')[['name']].rename({'name': 'country'}, axis=1)